## API

In [206]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [271]:
url = "https://api.coindesk.com/v1/bpi/historical/close.json"

def request(currency="USD",sd=None,ed=None):
    """The currency must be in ISO 4217 format. The Start Date (sd) and the End date (ed) must be in the format YYYY-MM-DD, 
      if you dont write any argument you will get the last 31 days """     
   
    try: 
        if sd==None and ed==None:
            response = requests.get(url+ "?currency="+currency)
        elif sd!=None and ed != None:
            response = requests.get(url+ "?currency="+ currency+ "&start=" + sd + "&end="+ed)

        results = response.json()
        df = pd.DataFrame(results)
        df.reset_index(level=0, inplace=True)
        df2 = df[["index","bpi"]]
        df2.columns = [["Date","Bitcoin Price {}".format(currency)]]
        df2.reset_index(level=0)
        return df2[:-2]
    except: 
        print("You must introduce both the sd and the ed of none of them, verify that the currency exists")
    

request("EUR","2015-10-10","2016-10-10")


,Date,Bitcoin Price EUR
0,2015-10-10,215.9811
1,2015-10-11,217.8799
2,2015-10-12,216.2337
3,2015-10-13,219.5886
4,2015-10-14,220.0079
5,2015-10-15,223.8958
6,2015-10-16,231.6831
7,2015-10-17,238.9689
8,2015-10-18,231.4955
9,2015-10-19,233.4014


In [247]:
response = requests.get('https://api.coindesk.com/v1/bpi/historical/close.json')
results = response.json()
df = pd.DataFrame(results)
df.reset_index(level=0, inplace=True)
df2 = df[["index","bpi"]]
df2.columns = [["Date","Bitcoin Price (USD)"]]
df3 = df2[:30]
df3.head()

,Date,Bitcoin Price (USD)
0,2018-10-01,6578.7938
1,2018-10-02,6506.6575
2,2018-10-03,6470.7138
3,2018-10-04,6555.8750
4,2018-10-05,6589.4038


In [ ]:
data = df3.to_csv('Output/API_Bitcoin.csv', index=False)

## Web Scrapping

In [178]:
url = ("https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html")
html = requests.get(url).content
result = BeautifulSoup(html,"lxml")


In [205]:
country_ = result.select(".currency")
country = [ctry.text for ctry in country_]
currency_ = result.select(".alignLeft")
currency = [curr.text for curr in currency_]
spot_ = result.select(".rate")
spot = [sp.text for sp in spot_]
dictionary = {"Country":country,"Currency":currency,"Spot":spot}
df = pd.DataFrame(dictionary)
df

,Country,Currency,Spot
0,USD,US dollar,1.1393
1,JPY,Japanese yen,128.52
2,BGN,Bulgarian lev,1.9558
3,CZK,Czech koruna,25.879
4,DKK,Danish krone,7.4604
5,GBP,Pound sterling,0.88173
6,HUF,Hungarian forint,322.96
7,PLN,Polish zloty,4.3269
8,RON,Romanian leu,4.6640
9,SEK,Swedish krona,10.3158


'USD'